# Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/DSAI Phase 2/Natural Language Processing/topical_chat.csv')

In [ ]:
data

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [ ]:
# Group messages by conversation ID
grouped_data = data.groupby('conversation_id')['message'].apply(list)

# Prepare pairs of questions and answers for each conversation
conversations = []
for _, messages in grouped_data.items():
    questions = messages[::2]  # Assuming questions are at even indices
    answers = messages[1::2]   # Assuming answers are at odd indices

    for question, answer in zip(questions, answers):
        conversations.append({'question': question, 'answer': answer})

# Print the first few conversations for verification
for i, conv in enumerate(conversations):
    print(f"Conversation {i + 1}:")
    print(f"Question: {conv['question']}")
    print(f"Answer: {conv['answer']}\n")
    if i >= 50:
        break

Conversation 1:
Question:  Are you a fan of Google or Microsoft?
Answer:  Both are excellent technology they are helpful in many ways. For the security purpose both are super.

Conversation 2:
Question:  I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. 
Answer:  Google provides online related services and products, which includes online ads, search engine and cloud computing.

Conversation 3:
Question:  Yeah, their services are good. I'm just not a fan of intrusive they can be on our personal lives. 
Answer:  Google is leading the alphabet subsidiary and will continue to be the Umbrella company for Alphabet internet interest.

Conversation 4:
Question:  Did you know Google had hundreds of live goats to cut the grass in the past? 
Answer:  It is very interesting. Google provide "Chrome OS" which is a light weight OS. Google provided a lot of hardware mainly in 2010 to 2015. 

Conversation 5:
Question:  I like Google Chrome. 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download necessary resources if not already downloaded
nltk.download('punkt_tab')

# Define a function to clean and preprocess the text
def clean_and_preprocess(text):
    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Join tokens back into a cleaned sentence
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Clean and preprocess the questions and answers in conversations
cleaned_conversations = []
for conv in conversations:
    cleaned_question = clean_and_preprocess(conv['question'])
    cleaned_answer = clean_and_preprocess(conv['answer'])
    cleaned_conversations.append({'question': cleaned_question, 'answer': cleaned_answer})

# Print the first few cleaned conversations for verification
for i, conv in enumerate(cleaned_conversations):
    print(f"Conversation {i + 1}:")
    print(f"Cleaned Question: {conv['question']}")
    print(f"Cleaned Answer: {conv['answer']}\n")
    if i >= 4:  # Print the first 5 cleaned conversations
     break

Conversation 1:
Cleaned Question: are you a fan of google or microsoft ?
Cleaned Answer: both are excellent technology they are helpful in many ways . for the security purpose both are super .

Conversation 2:
Cleaned Question: i 'm not a huge fan of google , but i use it a lot because i have to . i think they are a monopoly in some sense .
Cleaned Answer: google provides online related services and products , which includes online ads , search engine and cloud computing .

Conversation 3:
Cleaned Question: yeah , their services are good . i 'm just not a fan of intrusive they can be on our personal lives .
Cleaned Answer: google is leading the alphabet subsidiary and will continue to be the umbrella company for alphabet internet interest .

Conversation 4:
Cleaned Question: did you know google had hundreds of live goats to cut the grass in the past ?
Cleaned Answer: it is very interesting . google provide `` chrome os '' which is a light weight os . google provided a lot of hardware m

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Conversation 1:
Cleaned Question: are you a fan of google or microsoft ?
Cleaned Answer: both are excellent technology they are helpful in many ways . for the security purpose both are super .

Conversation 2:
Cleaned Question: i 'm not a huge fan of google , but i use it a lot because i have to . i think they are a monopoly in some sense .
Cleaned Answer: google provides online related services and products , which includes online ads , search engine and cloud computing .

Conversation 3:
Cleaned Question: yeah , their services are good . i 'm just not a fan of intrusive they can be on our personal lives .
Cleaned Answer: google is leading the alphabet subsidiary and will continue to be the umbrella company for alphabet internet interest .

Conversation 4:
Cleaned Question: did you know google had hundreds of live goats to cut the grass in the past ?
Cleaned Answer: it is very interesting . google provide `` chrome os '' which is a light weight os . google provided a lot of hardware m

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

# Combine all cleaned questions and answers into separate lists
questions = [conv['question'] for conv in cleaned_conversations]
answers = [conv['answer'] for conv in cleaned_conversations]

# Initialize tokenizers for questions and answers
tokenizer_ques = Tokenizer()  # Use <OOV> for out-of-vocabulary words
tokenizer_ans = Tokenizer()  # Use <OOV> for out-of-vocabulary words

# Fit the tokenizers on the text for questions and answers
tokenizer_ques.fit_on_texts(questions)
tokenizer_ans.fit_on_texts(answers)

# Add '<start>' and '<end>' tokens to the tokenizers' word_index for answers
tokenizer_ans.word_index['<start>'] = len(tokenizer_ans.word_index) + 1
tokenizer_ans.word_index['<end>'] = len(tokenizer_ans.word_index) + 2

# Save the tokenizers to files
with open('tokenizer_ques.pkl', 'wb') as tokenizer_ques_file:
    pickle.dump(tokenizer_ques, tokenizer_ques_file)
with open('tokenizer_ans.pkl', 'wb') as tokenizer_ans_file:
    pickle.dump(tokenizer_ans, tokenizer_ans_file)

# Convert text to sequences of word indices for questions and answers
sequences_ques = tokenizer_ques.texts_to_sequences(questions)
sequences_ans = tokenizer_ans.texts_to_sequences(answers)

# Find the maximum sequence length for questions and answers separately
max_seq_length_ques = max(len(seq) for seq in sequences_ques)
max_seq_length_ans = max(len(seq) for seq in sequences_ans)

# Pad sequences to make them of the same length for questions and answers
padded_sequences_ques = pad_sequences(sequences_ques, maxlen=max_seq_length_ques, padding='post', truncating='post')
padded_sequences_ans = pad_sequences(sequences_ans, maxlen=max_seq_length_ans, padding='post', truncating='post')

# Create input-output pairs for the encoder-decoder model for questions and answers
input_data_ques = padded_sequences_ques[:, :-1]  # Input is the question (remove the last token)
output_data_ans = padded_sequences_ans[:, 1:]   # Output is the answer (remove the first token)

# Convert input and output sequences to numpy arrays for questions and answers
input_data_ques = np.array(input_data_ques)
output_data_ans = np.array(output_data_ans)

# Print the vocabulary size for questions and answers separately
vocab_size_ques = len(tokenizer_ques.word_index)
vocab_size_ans = len(tokenizer_ans.word_index)
print(f"Vocabulary size for questions: {vocab_size_ques}")
print(f"Vocabulary size for answers: {vocab_size_ans}")

Vocabulary size for questions: 27213
Vocabulary size for answers: 28171


In [ ]:
max_seq_length_ques

130

In [ ]:
max_seq_length_ans - 2

130

In [ ]:
import pickle

# Create a dictionary to store the preprocessed data
preprocessed_data = {
    'input_data': input_data_ques,
    'output_data': output_data_ans,
    'tokenizer_ques': tokenizer_ques,  # Use tokenizer_ques
    'tokenizer_ans': tokenizer_ans,    # Use tokenizer_ans
    'max_seq_length_ques': max_seq_length_ques,  # Update variable names
    'max_seq_length_ans': max_seq_length_ans,    # Update variable names
    'vocab_size_ques': vocab_size_ques,  # Update variable names
    'vocab_size_ans': vocab_size_ans     # Update variable names
}

# Save the preprocessed data to a file using Pickle
with open('preprocessed_data.pkl', 'wb') as file:
    pickle.dump(preprocessed_data, file)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_output_train, decoder_output_val = train_test_split(
    input_data_ques, output_data_ans, output_data_ans, test_size=0.2, random_state=42
)

# Print the shapes of the training and validation sets
print("Shapes of training data:")
print(f"Encoder input: {encoder_input_train.shape}")
print(f"Decoder input: {decoder_input_train.shape}")
print(f"Decoder output: {decoder_output_train.shape}")

print("\nShapes of validation data:")
print(f"Encoder input: {encoder_input_val.shape}")
print(f"Decoder input: {decoder_input_val.shape}")
print(f"Decoder output: {decoder_output_val.shape}")


Shapes of training data:
Encoder input: (72939, 129)
Decoder input: (72939, 131)
Decoder output: (72939, 131)

Shapes of validation data:
Encoder input: (18235, 129)
Decoder input: (18235, 131)
Decoder output: (18235, 131)


In [ ]:
import tensorflow as tf

def batch_one_hot_encode(sequences, vocab_size, batch_size=8, use_embedding=True, embedding_dim=64):
    # Initialize list for encoded sequences
    encoded_sequences = []

    if use_embedding:
        # Use embedding layer for more efficient memory usage
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        embedded_sequences = embedding_layer(sequences)
        return embedded_sequences

    # Process smaller chunks to prevent memory overload
    with tf.device('/CPU:0'):  # Execute on CPU if GPU memory is a concern
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i : i + batch_size]
            encoded_batch = tf.one_hot(batch, depth=vocab_size)
            encoded_sequences.append(encoded_batch)

        return tf.concat(encoded_sequences, axis=0)

# Example usage
# Note: Decrease vocab_size if possible or switch to embeddings
decoder_output_train_encoded = batch_one_hot_encode(output_data_ans, vocab_size_ans, batch_size=8, use_embedding=True)


In [ ]:
import pickle
import numpy as np
import tensorflow as tf

# Load the tokenizers
with open('tokenizer_ques.pkl', 'rb') as tokenizer_ques_file:
    tokenizer_ques = pickle.load(tokenizer_ques_file)

with open('tokenizer_ans.pkl', 'rb') as tokenizer_ans_file:
    tokenizer_ans = pickle.load(tokenizer_ans_file)

# Load or define your sequences
# sequences_ques = load_sequences('encoder_sequences.pkl')
# sequences_ans = load_sequences('decoder_sequences.pkl')

# Pad sequences to ensure uniform shape
padded_sequences_ques = tf.keras.preprocessing.sequence.pad_sequences(sequences_ques, padding='post')
padded_sequences_ans = tf.keras.preprocessing.sequence.pad_sequences(sequences_ans, padding='post')

# Verify the shapes of your data
print("Shapes of your data:")
print("Encoder input:", np.array(padded_sequences_ques).shape)
print("Decoder input:", np.array(padded_sequences_ans).shape)

# Sample inspection
for i in range(5):  # Print the first 5 samples
    print("\nSample", i + 1)
    print("Encoder Input:", tokenizer_ques.sequences_to_texts([sequences_ques[i]]))
    print("Decoder Input:", tokenizer_ans.sequences_to_texts([sequences_ans[i]]))



Shapes of your data:
Encoder input: (91174, 130)
Decoder input: (91174, 132)

Sample 1
Encoder Input: ['are you a fan of google or microsoft']
Decoder Input: ['both are excellent technology they are helpful in many ways for the security purpose both are super']

Sample 2
Encoder Input: ["i 'm not a huge fan of google but i use it a lot because i have to i think they are a monopoly in some sense"]
Decoder Input: ['google provides online related services and products which includes online ads search engine and cloud computing']

Sample 3
Encoder Input: ["yeah their services are good i 'm just not a fan of intrusive they can be on our personal lives"]
Decoder Input: ['google is leading the alphabet subsidiary and will continue to be the umbrella company for alphabet internet interest']

Sample 4
Encoder Input: ['did you know google had hundreds of live goats to cut the grass in the past']
Decoder Input: ["it is very interesting google provide chrome os '' which is a light weight os google

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a custom dataset class
class MyDataset(tf.keras.utils.Sequence):
    def __init__(self, encoder_input, decoder_input, decoder_output, tknizer_ques, tknizer_ans, max_len):
        self.encoder_input = encoder_input.tolist()
        self.decoder_input = decoder_input.tolist()
        self.decoder_output = decoder_output.tolist()
        self.tknizer_ques = tknizer_ques
        self.tknizer_ans = tknizer_ans
        self.max_len = max_len

    def __len__(self):
        return len(self.encoder_input)

    def __getitem__(self, i):
        encoder_input = self.encoder_input[i]
        decoder_input = self.decoder_input[i]
        decoder_output = self.decoder_output[i]

        encoder_seq = self.tknizer_ques.texts_to_sequences([encoder_input])[0]
        decoder_inp_seq = self.tknizer_ans.texts_to_sequences([decoder_input])[0]
        decoder_out_seq = self.tknizer_ans.texts_to_sequences([decoder_output])[0]

        encoder_seq = pad_sequences([encoder_seq], maxlen=self.max_len, padding='post')[0]
        decoder_inp_seq = pad_sequences([decoder_inp_seq], maxlen=self.max_len, padding='post')[0]
        decoder_out_seq = pad_sequences([decoder_out_seq], maxlen=self.max_len, padding='post')[0]

        # Return data as tf.Tensor
        return (tf.convert_to_tensor([encoder_seq, decoder_inp_seq], dtype=tf.int32),
                tf.convert_to_tensor(decoder_out_seq, dtype=tf.int32))

    def on_epoch_end(self):
        indices = np.arange(len(self.encoder_input))
        np.random.shuffle(indices)
        self.encoder_input = [self.encoder_input[i] for i in indices]
        self.decoder_input = [self.decoder_input[i] for i in indices]
        self.decoder_output = [self.decoder_output[i] for i in indices]


In [ ]:
# Define a custom data loader class
class Dataloder(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset))

    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = [self.dataset[j] for j in range(start, stop)]

        encoder_seqs = np.stack([item[0][0] for item in data], axis=0)
        decoder_inp_seqs = np.stack([item[0][1] for item in data], axis=0)
        decoder_out_seqs = np.stack([item[1] for item in data], axis=0)

        # Convert data to tf.Tensor
        return (tf.convert_to_tensor(encoder_seqs, dtype=tf.int32),
                tf.convert_to_tensor(decoder_inp_seqs, dtype=tf.int32)), tf.convert_to_tensor(decoder_out_seqs, dtype=tf.int32)

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad your sequences to the max sequence length
encoder_input_train = pad_sequences(encoder_input_train, maxlen=max_seq_length_ques, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen=max_seq_length_ques, padding='post')
decoder_output_train = pad_sequences(decoder_output_train, maxlen=max_seq_length_ques, padding='post')

encoder_input_val = pad_sequences(encoder_input_val, maxlen=max_seq_length_ques, padding='post')
decoder_input_val = pad_sequences(decoder_input_val, maxlen=max_seq_length_ques, padding='post')
decoder_output_val = pad_sequences(decoder_output_val, maxlen=max_seq_length_ques, padding='post')


In [ ]:
# Step 9: Define the Encoder class
class Encoder(tf.keras.Model):
    def __init__(self, inp_vocab_size, embedding_size, lstm_size, input_length):
        super().__init__()
        self.embedding = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length, mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self, input_sequence, training=True):
        input_embedd = self.embedding(input_sequence)
        lstm_output, lstm_state_h, lstm_state_c = self.lstm(input_embedd)
        return lstm_output, lstm_state_h, lstm_state_c

    def initialize_states(self, batch_size):
        hidden_state = np.zeros((batch_size, self.lstm_size))
        cell_state = np.zeros((batch_size, self.lstm_size))
        return hidden_state, cell_state

In [ ]:
# Step 10: Define the Decoder class
class Decoder(tf.keras.Model):
    def __init__(self, out_vocab_size, embedding_size, lstm_size, input_length):
        super().__init__()
        self.embedding = Embedding(input_dim=out_vocab_size, output_dim=embedding_size, input_length=input_length, mask_zero=True, name="embedding_layer_decoder", trainable=False)
        self.lstm = LSTM(lstm_size, return_sequences=True, return_state=True, name="Decoder_LSTM")

    def call(self, input_sequence, states):
        embedd = self.embedding(input_sequence)
        lstm_output, final_state_h, final_state_c = self.lstm(embedd, initial_state=states)
        return lstm_output, final_state_h, final_state_c

In [ ]:
# Step 11: Define the Encoder-Decoder class
class EncoderDecoder(tf.keras.Model):
    def __init__(self, encoder_inputs_length, decoder_inputs_length, output_vocab_size):
        super().__init__()
        self.encoder = Encoder(inp_vocab_size=vocab_size_ques+1, embedding_size=100, lstm_size=512, input_length=encoder_inputs_length)
        self.decoder = Decoder(out_vocab_size=vocab_size_ans+1, embedding_size=100, lstm_size=512, input_length=decoder_inputs_length)
        self.dense = Dense(output_vocab_size, activation='softmax')

    def call(self, data):
        encoder_output, encoder_h, encoder_c = self.encoder(data[0])
        decode, _, _ = self.decoder(data[1], [encoder_h, encoder_c])
        decoder_outputs = self.dense(decode)
        return decoder_outputs

In [ ]:
# Step 13: Create training and validation datasets
train_dataset = MyDataset(encoder_input_train, decoder_input_train, decoder_output_train, tokenizer_ques, tokenizer_ans, max_seq_length_ques-1)
val_dataset = MyDataset(encoder_input_val, decoder_input_val, decoder_output_val, tokenizer_ques, tokenizer_ans, max_seq_length_ques-1)

train_dataloader = Dataloder(train_dataset, batch_size=32)
val_dataloader = Dataloder(val_dataset, batch_size=32)

In [ ]:
# Step 15: Inspect training data shapes
print("Shapes of training data:")
print(f"Encoder input: {encoder_input_train.shape}")
print(f"Decoder input: {decoder_input_train.shape}")
print(f"Decoder output: {decoder_output_train.shape}")

Shapes of training data:
Encoder input: (72939, 129)
Decoder input: (72939, 131)
Decoder output: (72939, 131)


In [ ]:
# Step 17: Define the training parameters
epochs = 10  # Adjust based on needs
batch_size = 32

# Convert custom dataloaders to tf.data.Dataset
def convert_to_tf_dataset(dataloader):
    dataset = tf.data.Dataset.from_generator(
        lambda: iter(dataloader),
        output_signature=(
            (tf.TensorSpec(shape=(None, max_seq_length_ques-1), dtype=tf.int32),
             tf.TensorSpec(shape=(None, max_seq_length_ans-1), dtype=tf.int32)),
            tf.TensorSpec(shape=(None, max_seq_length_ans-1), dtype=tf.int32)
        )
    )
    return dataset

In [ ]:
# Create tf.data.Dataset from the custom dataloaders
train_dataset = convert_to_tf_dataset(train_dataloader).batch(batch_size)
val_dataset = convert_to_tf_dataset(val_dataloader).batch(batch_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(input_vocab_size, output_vocab_size, embedding_dim=64, lstm_units=256, max_seq_length=20):
    encoder_input = layers.Input(shape=(max_seq_length,))
    decoder_input = layers.Input(shape=(max_seq_length,))

    encoder_embedded = layers.Embedding(input_vocab_size, embedding_dim)(encoder_input)
    encoder_lstm = layers.LSTM(lstm_units, return_state=True)(encoder_embedded)
    encoder_output, state_h, state_c = encoder_lstm

    decoder_embedded = layers.Embedding(output_vocab_size, embedding_dim)(decoder_input)
    decoder_lstm = layers.LSTM(lstm_units, return_sequences=True)(decoder_embedded, initial_state=[state_h, state_c])
    decoder_output = layers.Dense(output_vocab_size, activation='softmax')(decoder_lstm)

    model = models.Model([encoder_input, decoder_input], decoder_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Example dataset creation (use your dataset here)
train_dataset = tf.data.Dataset.from_tensor_slices((
    (encoder_input_train, decoder_input_train), decoder_output_train
))
train_dataset = train_dataset.batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    (encoder_input_val, decoder_input_val), decoder_output_val
))
val_dataset = val_dataset.batch(8)

# Create the model
input_vocab_size = len(tokenizer_ques.word_index) + 1
output_vocab_size = len(tokenizer_ans.word_index) + 1
model = create_model(input_vocab_size, output_vocab_size, max_seq_length=max_seq_length_ques)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)


Epoch 1/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 771s 84ms/step - accuracy: 0.9166 - loss: 0.6999 - val_accuracy: 0.9928 - val_loss: 0.0663
Epoch 2/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 766s 84ms/step - accuracy: 0.9941 - loss: 0.0500 - val_accuracy: 0.9966 - val_loss: 0.0320
Epoch 3/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 767s 84ms/step - accuracy: 0.9974 - loss: 0.0177 - val_accuracy: 0.9973 - val_loss: 0.0276
Epoch 4/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 805s 85ms/step - accuracy: 0.9985 - loss: 0.0072 - val_accuracy: 0.9975 - val_loss: 0.0278
Epoch 5/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 766s 84ms/step - accuracy: 0.9993 - loss: 0.0029 - val_accuracy: 0.9976 - val_loss: 0.0270
Epoch 6/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 765s 84ms/step - accuracy: 0.9997 - loss: 0.0010 - val_accuracy: 0.9977 - val_loss: 0.0253
Epoch 7/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 765s 84ms/step - accuracy: 0.9999 - loss: 4.0095e-04 - val_accuracy: 0.9978 - val_loss: 0.0229
Epoch 8/10
9118/9118 ━━━━━━━━━━━━━━━━━━━━ 804s 84ms/step - accura

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your predictions and true labels are stored as 'y_pred' and 'y_true'
# Replace 'y_pred' and 'y_true' with the actual variables
y_true = np.array(decoder_output_val)  # True labels from validation
y_pred = np.array(val_dataset)  # Predicted labels from model

# Confusion Matrix
cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=tokenizer_ans.word_index.keys(), yticklabels=tokenizer_ans.word_index.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 21992 (\N{CJK UNIFIED IDEOGRAPH-55E8}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:648: UserWarning: Glyph 21992 (\N{CJK UNIFIED IDEOGRAPH-55E8}) missing from current font.
  bboxes = [l.get_window_extent() for l in labels]


In [ ]:
# F1 Score
f1 = f1_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print(f'F1 Score: {f1}')

# Accuracy Score
accuracy = accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
print(f'Accuracy: {accuracy}')


NameError: name 'y_pred' is not defined

In [ ]:

# Classification Report
print("Classification Report:")
print(classification_report(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
# Assuming 'history' is the training history returned from model.fit()
history = model.fit(...)

# Plot Training & Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot Training & Validation Accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
